# Introduction

The objective of this project is to analyze and build a Machine Learning model based on customer churn data. The dataset contains information about a fictional telecommunications company that provided landline phone and internet services to 7,043 customers in California in the third quarter. It indicates which customers left, stayed, or signed up for their service. Churn prediction identifies customers who are likely to cancel their contracts soon. If the company can do this, it can address the users before churn.

The target variable we want to predict is categorical and has only two possible outcomes: churn or non-churn (Binary Classification). We would also like to understand why the model thinks our customers are leaving, and for that, we need to be able to interpret the model's predictions. According to the description, this dataset contains the following information:

* Customer services: phone; multiple lines; internet; tech support and extra services, such as online security, backup, device protection, and TV streaming
* Account information: how long they've been a customer, contract type, payment method type
* Charges: how much was charged to the customer last month and in total
* Demographic information: gender, age, and whether they have dependents or a partner
* Churn: yes/no, whether the customer left the company last month

# Project Structure

The development of this project followed a structured and iterative methodology, covering everything from data collection and preparation to the evaluation and interpretation of results.

In [ ]:
#| include: false
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, RocCurveDisplay, make_scorer, recall_score, precision_recall_curve, auc
from imblearn.combine import SMOTETomek
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from IPython.display import display
import lightgbm as lgb
import shap
import joblib
import warnings
# Suprimir avisos para uma saída mais limpa, se necessário (ex: de versões antigas do XGBoost)
warnings.filterwarnings('ignore')

In [ ]:
#| include: false
df = pd.read_csv('dataset-churn.csv')

# Exploratory Data Analysis (EDA)

###### Columns:

In [ ]:
df.columns

###### Sample:

In [ ]:
df.sample(5)

###### Info:

In [ ]:
df.info()

The column TotalCharges is currently an object and needs to be converted to a numeric type.
###### Info:

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

###### Describe:

In [ ]:
df.describe()

###### Checking Missing Values:

In [ ]:
print(df.isnull().sum())

The TotalCharges column has 11 missing values. Let's analyze it to define the best strategy to handle them.

###### Displaying the Full Rows with Missing Value:

In [ ]:
df[df['TotalCharges'].isna()]

We observe that the rows with missing TotalCharges have a tenure of 0. This means they are new customers who haven't accumulated any charges yet. Therefore, we will impute these missing values with 0.

###### Filling the missing values with 0:

In [ ]:
df['TotalCharges'] = df['TotalCharges'].fillna(0)

###### Checking Missing Values Again:

In [ ]:
df.isnull().sum()

###### TotalCharges Distribution:

In [ ]:
plt.figure(figsize = (6, 6))
plt.title("TotalCharges")
sns.boxplot(data = df['TotalCharges'], color = 'blue')
plt.show()

## Univariate Analysis

###### TotalCharges:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('TotalCharges Analysis', fontsize=16)

sns.boxplot(data=df['TotalCharges'], color='blue', ax=axes[0])
axes[0].set_title('Boxplot')

sns.histplot(df['TotalCharges'], color='red', kde=True, ax=axes[1])
axes[1].set_title('TotalCharges Distribution')
axes[1].set_ylabel('Density')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

###### MonthlyCharges:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('MonthlyCharges Analysis', fontsize=16)

sns.boxplot(data=df['MonthlyCharges'], color='blue', ax=axes[0])
axes[0].set_title('Boxplot')

sns.histplot(df['MonthlyCharges'], color='red', kde=True, ax=axes[1])
axes[1].set_title('MonthlyCharges Distribution')
axes[1].set_ylabel('Density')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

###### Tenure:

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Tenure Analysis', fontsize=16)

sns.boxplot(data=df['tenure'], color='blue', ax=axes[0])
axes[0].set_title('Boxplot')

sns.histplot(df['tenure'], color='red', kde=True, ax=axes[1])
axes[1].set_title('Tenure Distribution')
axes[1].set_ylabel('Density')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

We are going to replace 'No internet service' and 'No phone service' with 'No'. This is because they carry the same meaning (the customer does not have that particular service), and keeping them as separate categories would create unnecessary features without adding relevant information for the model.

###### Replacing these values with "No" across the DataFrame:

In [ ]:
valores_substituir = ["No internet service", "No phone service"]
df = df.replace(valores_substituir, "No")

Let's analyze the churn rate across our categorical variables, comparing the rate for each category against the global average churn rate

###### Creating Visuals:

In [ ]:
df_copy = df.copy()
df_copy['Churn'] = df_copy['Churn'].map({'No': 0, 'Yes': 1})
categorical_features = [
    'gender', 'MultipleLines', 'InternetService', 'OnlineSecurity',
    'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
    'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod'
]
global_mean = df_copy['Churn'].mean()
print(f"Global Mean Churn Rate: {global_mean:.2f}")

n_features = len(categorical_features)
ncols = 1
nrows = (n_features + ncols - 1) // ncols

fig, axes = plt.subplots(nrows, ncols, figsize=(12, nrows * 5), constrained_layout=True)
axes = axes.flatten() if isinstance(axes, (list, np.ndarray)) else [axes]

for i, feature in enumerate(categorical_features):
    df_group = df_copy.groupby(by=feature)['Churn'].agg(['mean']).reset_index()
    ax = axes[i]
    graph = sns.barplot(x=feature, y='mean', data=df_group, palette='viridis', ax=ax)
    ax.axhline(global_mean, linewidth=2, color='red', linestyle='--')
    ax.text(0, global_mean + 0.01, f"Global Mean: {global_mean:.2f}", color='red', weight='semibold')
    ax.set_title(f"Churn Rate by {feature}", fontsize=14)
    ax.set_ylabel("Mean Churn Rate", fontsize=12)
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylim(0, max(df_group['mean'].max() * 1.2, global_mean * 1.2))
    ax.tick_params(axis='x', labelrotation=0)
for j in range(i + 1, len(axes)):
    axes[j].axis('off')
plt.show()

## Bivariate Analysis

###### MonthlyCharges x Churn:

In [ ]:
plt.figure(figsize = (12, 5))
sns.kdeplot(data=df, x='MonthlyCharges', hue='Churn', fill=True, common_norm=False, palette='mako')
plt.show()

###### TotalCharges x Churn:

In [ ]:
plt.figure(figsize = (12, 5))
sns.kdeplot(data=df, x='TotalCharges', hue='Churn', fill=True, common_norm=False, palette='mako')
plt.show()

###### Tenure x Churn:

In [ ]:
plt.figure(figsize = (12, 5))
sns.kdeplot(data=df, x='tenure', hue='Churn', fill=True, common_norm=False, palette='mako')
plt.show()

### Inights

The average churn rate found in the dataset is approximately 27%, meaning that roughly one in every four customers terminates their contract. This is a significant rate and should be treated as a critical retention indicator. An analysis of the tenure variable shows a median of 29 months, but with 25% of customers staying for nine months or less, revealing a high concentration of short-term customers and suggesting that early churn is a primary risk.

Regarding billing, the MonthlyCharges variable has a mean of around 64.8 but with a wide spread, indicating highly distinct consumption profiles. TotalCharges, in turn, reflects both the variation in contract length and the accumulation of spending over time.

Observing the categorical variables, it's notable that customers on month-to-month contracts, those who use paperless billing, and those with certain payment methods exhibit churn rates higher than the global average. These profiles, therefore, constitute strategic targets for targeted interventions. It is also worth highlighting that the data handling applied to TotalCharges was essential to prevent distortions and ensure consistency in the comparisons made.

The relationship between churn and tenure reveals that customers who cancel tend to be concentrated in the first few months of their contract, reinforcing the need for specific onboarding and loyalty strategies in the very first cycles. The comparison between churn and MonthlyCharges indicates that there are billing ranges in which the propensity to cancel is higher, especially among customers with higher invoice values, which suggests a need to evaluate the perceived cost-benefit in this segment.

# Data Preprocessing

###### Dropping the customerID column as it has no predictive value:

In [ ]:
df = df.drop('customerID', axis=1) 

###### Describe current df:

In [ ]:
df.describe(include=['object', 'bool'])

###### Label Encoding for binary variables:

In [ ]:
binarias = [
    'gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn'
]
# Mapping Yes/No to 1/0 and Male/Female to 1/0
mapeamento = {'Yes': 1,'No': 0,'Male': 1,'Female': 0}
for col in binarias:
    df[col] = df[col].map(mapeamento)

###### One-Hot Encoding for nominal variable:

In [ ]:
nominais = ['InternetService', 'Contract', 'PaymentMethod']
df = pd.get_dummies(df, columns=nominais, drop_first=True, dtype=int)

###### Correlation Analysis of Numerical Variables:

In [ ]:
sns.heatmap(df[['tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']].corr(), annot=True, cmap='coolwarm')

###### Correlation Analysis with the 'Churn' Column:

In [ ]:
corr_matrix = df.corr()
corr_churn = corr_matrix['Churn'].sort_values(ascending=False)
print(corr_churn)

###### Correlation Matrix:

In [ ]:
corr_matrix = df.corr()
limite = 0.7
corr_pairs = (
    corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    .stack()
    .reset_index()
)
corr_pairs.columns = ['Var1', 'Var2', 'Correlation']
corr_high = corr_pairs[abs(corr_pairs['Correlation']) > limite].sort_values(
    by='Correlation', ascending=False
)
print("Highly correlated variables (|correlation| > 0.7):")
print(corr_high)
corr_filtrada = corr_matrix.mask(abs(corr_matrix) <= limite)

The 0.826 correlation between tenure and TotalCharges is extremely high, indicating strong multicollinearity. TotalCharges is fundamentally the product of tenure and MonthlyCharges (or, more precisely, the sum of MonthlyCharges over the tenure). Keeping all three variables together would add redundancy and could complicate interpretability, especially for linear models, without necessarily adding more predictive power.

We opted to remove TotalCharges, as we still retain crucial information from tenure (the customer's length of stay, a key indicator of loyalty and maturity) and MonthlyCharges (the current cost of service, which correlates with the quality/service package and is a strong predictor of churn). Together, these two variables already capture the essence of what TotalCharges represents.

The other strong correlations reflect the intrinsic business relationship between the service value (MonthlyCharges) and the service type (InternetService). Removing either of these variables would mean losing valuable information about the customer's service package, which is a very strong predictor of churn. The Machine Learning model can and should use this information to learn the relationship. The multicollinearity here is more of a reflection that one variable (MonthlyCharges) is heavily influenced by another (InternetService), and both are relevant.

###### Dropping the TotalCharges column:

In [ ]:
df = df.drop('TotalCharges', axis=1) 

###### Distribution of the Target Variable:

In [ ]:
churn_counts = df['Churn'].value_counts()
print("Churn Count:")
print(churn_counts)
plt.figure(figsize=(6,6))
plt.pie(
    churn_counts, 
    labels=['No (0)', 'Yes (1)'],   
    autopct='%1.1f%%', 
    startangle=90, 
    colors=['#1f77b4', '#ff7f0e'],
    explode=(0,0.05),
    shadow=True
)
plt.title("Churn Distribution (%)")
plt.show() 

We are facing a class imbalance problem in our target variable. To address this, we will test different strategies and approaches to determine which one yields the best model. Our objective is to build a model with an accuracy above 80%.

Why Recall? In the context of customer churn prediction, Recall (also known as Sensitivity or True Positive Rate) is the primary metric to optimize. It measures our model's ability to correctly identify the highest possible proportion of customers who will actually leave the company (the "Churn" minority class).

Minimizing Customer Losses (False Negatives): The cost of a False Negative (a customer the model predicted would not churn, but who actually did) is typically very high for a business. Each lost customer represents not only the interruption of future revenue but also the potential loss of Lifetime Value (LTV), the costs of acquiring new customers, and a negative impact on reputation. A high Recall means we are minimizing these False Negatives, ensuring that most customers at risk of churning are flagged for intervention.

Opportunity for Proactive Intervention: By identifying a customer with a high probability of churning (thanks to a high Recall), the company gains the opportunity to implement targeted retention strategies (personalized offers, proactive support, satisfaction surveys). Missing this opportunity due to a False Negative is the most costly error in this scenario.

Although Recall is the priority, we do not disregard Precision, F1-Score, and AUC-ROC:

In summary, our focus is to ensure that the largest possible number of at-risk customers are identified (high Recall), enabling effective retention actions. The other metrics help us refine the model, ensuring that these interventions are as efficient and targeted as possible, thus optimizing the return on investment (ROI) in churn prevention.

###### Separating Features (X) and Target Variable (y):

In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

###### Splitting data into train and test:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

###### Scaling Numerical Variables:

In [ ]:
numeric_cols = ['tenure', 'MonthlyCharges']
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

###### Visualizing the variables after scaling:

In [ ]:
numeric_cols = ["tenure", 'MonthlyCharges']
for feat in numeric_cols:
    plt.figure(figsize=(8, 5)) 
    sns.histplot(data=X_train, x=feat, kde=True)
    plt.title(f'{feat} Distribution', fontsize=14)
    plt.show()

# Building and Evaluating Machine Learning Models

## Logistic Regression:

In [ ]:
log_reg_model = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42, max_iter=1000)
log_reg_model.fit(X_train, y_train)
y_pred_log_reg = log_reg_model.predict(X_test)
y_proba_log_reg = log_reg_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Logistic Regression) ---")
print(classification_report(y_test, y_pred_log_reg))

cm_log_reg = confusion_matrix(y_test, y_pred_log_reg)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_log_reg, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Real')
plt.xlabel('Predict')
plt.title('Confusion Matrix')
plt.show()
# AUC-ROC
auc_log_reg = roc_auc_score(y_test, y_proba_log_reg)
print(f"\nAUC-ROC (Logistic Regression): {auc_log_reg:.4f}")
# Plotar Curva ROC
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(log_reg_model, X_test, y_test, ax=ax, name='Logistic Regression')
plt.title('ROC Curve - (Logistic Regression)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## Random Forest:

In [ ]:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)
y_proba_rf = rf_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Random Forest) ---")
print(classification_report(y_test, y_pred_rf))

cm_rf = confusion_matrix(y_test, y_pred_rf)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Real')
plt.xlabel('Predict')
plt.title('Confusion Matrix - Random Forest')
plt.show()
# Calcular e exibir AUC-ROC
auc_rf = roc_auc_score(y_test, y_proba_rf)
print(f"\nAUC-ROC (Random Forest): {auc_rf:.4f}")

# Plotar Curva ROC
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(rf_model, X_test, y_test, ax=ax, name='Random Forest')
plt.title('ROC Curve - Random Forest')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

### Using Class Balancing

To address class imbalance, a hybrid resampling approach combining SMOTE with undersampling is highly effective. SMOTE (Synthetic Minority Over-sampling Technique) avoids overfitting by creating new, synthetic samples for the minority class based on its nearest neighbors, rather than just duplicating data. This is then coupled with an undersampling technique, such as Tomek Links (as in SMOTETomek), which cleans the feature space by removing majority class samples that are close to the newly created synthetic minority points. This dual strategy results in a more balanced and less noisy dataset, allowing the model to learn a clearer and more robust decision boundary between the classes.

### Applying SMOTE + Undersampling (SMOTETomek)

In [ ]:
smote_tomek = SMOTETomek(random_state=42)
X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train)

print("\nClass Distribution After Applying SMOTETomek")
print(y_train_resampled.value_counts())

## Logistic Regression with SMOTETomek:

In [ ]:
log_reg_smote_model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000) # SEM class_weight='balanced'
log_reg_smote_model.fit(X_train_resampled, y_train_resampled)
y_pred_log_reg_smote = log_reg_smote_model.predict(X_test)
y_proba_log_reg_smote = log_reg_smote_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Logistic Regression with SMOTETomek) ---")
print(classification_report(y_test, y_pred_log_reg_smote))

cm_log_reg_smote = confusion_matrix(y_test, y_pred_log_reg_smote)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_log_reg_smote, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Real')
plt.xlabel('Predict')
plt.title('Confusion Matrix (Logistic Regression with SMOTETomek)')
plt.show()

auc_log_reg_smote = roc_auc_score(y_test, y_proba_log_reg_smote)
print(f"\nAUC-ROC (Logistic Regression with SMOTETomek): {auc_log_reg_smote:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(log_reg_smote_model, X_test, y_test, ax=ax, name='Logistic Regression (SMOTETomek)')
plt.title('ROC Curve - Logistic Regression (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## Random Forest with SMOTETomek

In [ ]:
rf_smote_model = RandomForestClassifier(random_state=42, n_estimators=100)
rf_smote_model.fit(X_train_resampled, y_train_resampled)
y_pred_rf_smote = rf_smote_model.predict(X_test)
y_proba_rf_smote = rf_smote_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Random Forest with SMOTETomek) ---")
print(classification_report(y_test, y_pred_rf_smote))

cm_rf_smote = confusion_matrix(y_test, y_pred_rf_smote)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf_smote, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Real')
plt.xlabel('Predict')
plt.title('Confusion Matrix - Random Forest (SMOTETomek)')
plt.show()

auc_rf_smote = roc_auc_score(y_test, y_proba_rf_smote)
print(f"\nAUC-ROC (Random Forest w/ SMOTETomek): {auc_rf_smote:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(rf_smote_model, X_test, y_test, ax=ax, name='Random Forest (SMOTETomek)')
plt.title('ROC Curve - Random Forest (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## SVM with SMOTETomek

In [ ]:
svm_model = SVC(kernel='rbf', probability=True, random_state=42)
svm_model.fit(X_train_resampled, y_train_resampled)
y_pred_svm = svm_model.predict(X_test)
y_proba_svm = svm_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (SVM with SMOTETomek) ---")
print(classification_report(y_test, y_pred_svm))

cm_svm = confusion_matrix(y_test, y_pred_svm)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - SVM (SMOTETomek)')
plt.show()

auc_svm = roc_auc_score(y_test, y_proba_svm)
print(f"\nAUC-ROC (SVM with SMOTETomek): {auc_svm:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(svm_model, X_test, y_test, ax=ax, name='SVM')
plt.title('ROC Curve - SVM (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## Decision Tree with SMOTETomek

In [ ]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_model.fit(X_train_resampled, y_train_resampled)
y_pred_dt = dt_model.predict(X_test)
y_proba_dt = dt_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Decision Tree with SMOTETomek) ---")
print(classification_report(y_test, y_pred_dt))

cm_dt = confusion_matrix(y_test, y_pred_dt)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_dt, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Decision Tree (SMOTETomek)')
plt.show()

auc_dt = roc_auc_score(y_test, y_proba_dt)
print(f"\nAUC-ROC (Decision Tree with SMOTETomek): {auc_dt:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(dt_model, X_test, y_test, ax=ax, name='Decision Tree')
plt.title('ROC Curve - Decision Tree (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## XGBoost with SMOTETomek

In [ ]:
xgb_model = XGBClassifier(objective='binary:logistic', eval_metric='logloss', use_label_encoder=False, random_state=42)
xgb_model.fit(X_train_resampled, y_train_resampled)
y_pred_xgb = xgb_model.predict(X_test)
y_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (XGBoost with SMOTETomek) ---")
print(classification_report(y_test, y_pred_xgb))

cm_xgb = confusion_matrix(y_test, y_pred_xgb)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - XGBoost (SMOTETomek)')
plt.show()

auc_xgb = roc_auc_score(y_test, y_proba_xgb)
print(f"\nAUC-ROC (XGBoost with SMOTETomek): {auc_xgb:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(xgb_model, X_test, y_test, ax=ax, name='XGBoost')
plt.title('ROC Curve - XGBoost (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## LightGBM with SMOTETomek

In [ ]:
lgbm_model = LGBMClassifier(objective='binary', metric='binary_logloss', random_state=42)
lgbm_model.fit(X_train_resampled, y_train_resampled)
y_pred_lgbm = lgbm_model.predict(X_test)
y_proba_lgbm = lgbm_model.predict_proba(X_test)[:, 1]


print("\n--- Classification Report (LightGBM with SMOTETomek) ---")
print(classification_report(y_test, y_pred_lgbm))

cm_lgbm = confusion_matrix(y_test, y_pred_lgbm)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_lgbm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - LightGBM (SMOTETomek)')
plt.show()

auc_lgbm = roc_auc_score(y_test, y_proba_lgbm)
print(f"\nAUC-ROC (LightGBM with SMOTETomek): {auc_lgbm:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(lgbm_model, X_test, y_test, ax=ax, name='LightGBM')
plt.title('ROC Curve - LightGBM (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## K-Nearest Neighbors (KNN) with SMOTETomek

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_resampled, y_train_resampled)
y_pred_knn = knn_model.predict(X_test)
y_proba_knn = knn_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (KNN with SMOTETomek) ---")
print(classification_report(y_test, y_pred_knn))

cm_knn = confusion_matrix(y_test, y_pred_knn)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_knn, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - KNN (SMOTETomek)')
plt.show()

auc_knn = roc_auc_score(y_test, y_proba_knn)
print(f"\nAUC-ROC (KNN with SMOTETomek): {auc_knn:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(knn_model, X_test, y_test, ax=ax, name='KNN')
plt.title('ROC Curve - KNN (SMOTETomek)')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

### Results Summary
| Model                     | Approach                   | Precision (Churn=1)| Recall (Churn=1) | F1-Score (Churn=1) | AUC-ROC |
| :------------------------- | :------------------------ | :----------------- | :--------------- | :----------------- | :------ |
| Logistic Regression        | `class_weight='balanced'` | 0.51               | **0.78**         | 0.62               | **0.8389**|
| Random Forest              | `class_weight='balanced'` | **0.63**           | 0.49             | 0.55               | 0.8195  |
| Logistic Regression        | SMOTETomek                | 0.53               | 0.74             | 0.62               | 0.8323  |
| Random Forest              | SMOTETomek                | 0.55               | 0.64             | 0.59               | 0.8167  |
| SVM                        | SMOTETomek                | 0.55               | 0.73             | 0.62               | 0.8233  |
| Decision Tree              | SMOTETomek                | 0.50               | 0.61             | 0.55               | 0.6958  |
| XGBoost                    | SMOTETomek                | 0.55               | 0.68             | 0.61               | 0.8169  |
| LightGBM                   | SMOTETomek                | 0.54               | 0.69             | 0.61               | 0.8268  |
| K-Nearest Neighbors (KNN)  | SMOTETomek                | 0.48               | **0.74**         | 0.58               | 0.7799  |

### Cross-Validation and Hyperparameter Optimization

In [ ]:
count_class_0 = y_train.value_counts()[0]
count_class_1 = y_train.value_counts()[1]
scale_pos_weight_value = count_class_0 / count_class_1
print(f"Number of No Churn (class 0) in training set: {count_class_0}")
print(f"Number of Churn (class 1) in training set: {count_class_1}")
print(f"Calculated scale_pos_weight: {scale_pos_weight_value:.4f}")
recall_scorer = make_scorer(recall_score, pos_label=1)

## Logistic Regression - Tunned

In [ ]:
print("\n--- Optimizing Hyperparameters for Logistic Regression (class_weight='balanced') ---")
log_reg_base = LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42, max_iter=1000)

param_grid_lr = {
    'C': np.logspace(-3, 2, 6)
}

grid_search_lr = GridSearchCV(estimator=log_reg_base,
                               param_grid=param_grid_lr,
                               scoring=recall_scorer,
                               cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1,
                               verbose=1)

grid_search_lr.fit(X_train, y_train)
print(f"Best Parameters (Logistic Regression): {grid_search_lr.best_params_}")
print(f"Best Recall (Logistic Regression) on CV: {grid_search_lr.best_score_:.4f}")
best_lr_model = grid_search_lr.best_estimator_
y_pred_lr_tuned = best_lr_model.predict(X_test)
y_proba_lr_tuned = best_lr_model.predict_proba(X_test)[:, 1]
print("\n--- Classification Report (Tuned Logistic Regression) ---")
print(classification_report(y_test, y_pred_lr_tuned))
cm_lr_tuned = confusion_matrix(y_test, y_pred_lr_tuned)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_lr_tuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Tuned Logistic Regression')
plt.show()

auc_lr_tuned = roc_auc_score(y_test, y_proba_lr_tuned)
print(f"AUC-ROC (Tuned Logistic Regression): {auc_lr_tuned:.4f}")
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_lr_model, X_test, y_test, ax=ax, name='Tuned Logistic Regression')
plt.title('ROC Curve - Tuned Logistic Regression')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## Random Forest - Tunned

In [ ]:
print("\n\n--- Optimizing Hyperparameters for Random Forest (class_weight='balanced') ---")

rf_base = RandomForestClassifier(class_weight='balanced', random_state=42)

param_grid_rf = {
    'n_estimators': [100, 200, 300], # Number of trees
    'max_depth': [None, 10, 20], # Maximum tree depth
    'min_samples_split': [2, 5, 10] # Minimum number of samples to split a node
}

grid_search_rf = GridSearchCV(estimator=rf_base,
                               param_grid=param_grid_rf,
                               scoring=recall_scorer,
                               cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1,
                               verbose=1)

grid_search_rf.fit(X_train, y_train)

print(f"Best Parameters (Random Forest): {grid_search_rf.best_params_}")
print(f"Best Recall (Random Forest) on CV: {grid_search_rf.best_score_:.4f}")

best_rf_model = grid_search_rf.best_estimator_
y_pred_rf_tuned = best_rf_model.predict(X_test)
y_proba_rf_tuned = best_rf_model.predict_proba(X_test)[:, 1]

print("\n--- Classification Report (Random Forest Tuned) ---")
print(classification_report(y_test, y_pred_rf_tuned))

cm_rf_tuned = confusion_matrix(y_test, y_pred_rf_tuned)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_rf_tuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Random Forest Tuned')
plt.show()

auc_rf_tuned = roc_auc_score(y_test, y_proba_rf_tuned)
print(f"AUC-ROC (Random Forest Tuned): {auc_rf_tuned:.4f}")

fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_rf_model, X_test, y_test, ax=ax, name='Random Forest Tuned')
plt.title('ROC Curve - Random Forest Tuned')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## Support Vector Machine (SVM) - Tunned

In [ ]:
print("\n\n--- Optimizing Hyperparameters for SVM (class_weight='balanced') ---")
svm_base = SVC(class_weight='balanced', probability=True, random_state=42)

param_grid_svm = {
    'C': [0.1, 1, 10],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['rbf']
}

grid_search_svm = GridSearchCV(estimator=svm_base,
                               param_grid=param_grid_svm,
                               scoring=recall_scorer,
                               cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1,
                               verbose=1)

grid_search_svm.fit(X_train, y_train)
print(f"Best Parameters (SVM): {grid_search_svm.best_params_}")
print(f"Best Recall (SVM) on CV: {grid_search_svm.best_score_:.4f}")

best_svm_model = grid_search_svm.best_estimator_
y_pred_svm_tuned = best_svm_model.predict(X_test)
y_proba_svm_tuned = best_svm_model.predict_proba(X_test)[:, 1]
print("\n--- Classification Report (SVM Tuned) ---")
print(classification_report(y_test, y_pred_svm_tuned))
cm_svm_tuned = confusion_matrix(y_test, y_pred_svm_tuned)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_svm_tuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - SVM Tuned')
plt.show()
auc_svm_tuned = roc_auc_score(y_test, y_proba_svm_tuned)
print(f"AUC-ROC (SVM Tuned): {auc_svm_tuned:.4f}")
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_svm_model, X_test, y_test, ax=ax, name='SVM Tuned')
plt.title('ROC Curve - SVM Tuned')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## XGBoost- Tunned

In [ ]:
print("\n\n--- Optimizing Hyperparameters for XGBoost (scale_pos_weight) ---")

xgb_base = XGBClassifier(objective='binary:logistic', eval_metric='logloss',
                         use_label_encoder=False, random_state=42,
                         scale_pos_weight=scale_pos_weight_value)
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

grid_search_xgb = GridSearchCV(estimator=xgb_base,
                               param_grid=param_grid_xgb,
                               scoring=recall_scorer,
                               cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1,
                               verbose=1)

grid_search_xgb.fit(X_train, y_train)
print(f"Best Parameters (XGBoost): {grid_search_xgb.best_params_}")
print(f"Best Recall (XGBoost) on CV: {grid_search_xgb.best_score_:.4f}")
best_xgb_model = grid_search_xgb.best_estimator_
y_pred_xgb_tuned = best_xgb_model.predict(X_test)
y_proba_xgb_tuned = best_xgb_model.predict_proba(X_test)[:, 1]
print("\n--- Classification Report (XGBoost Tuned) ---")
print(classification_report(y_test, y_pred_xgb_tuned))
cm_xgb_tuned = confusion_matrix(y_test, y_pred_xgb_tuned)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_xgb_tuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - XGBoost Tuned')
plt.show()
auc_xgb_tuned = roc_auc_score(y_test, y_proba_xgb_tuned)
print(f"AUC-ROC (XGBoost Tuned): {auc_xgb_tuned:.4f}")
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_xgb_model, X_test, y_test, ax=ax, name='XGBoost Tuned')
plt.title('ROC Curve - XGBoost Tuned')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

## LightGBM - Tunned

In [ ]:
print("\n\n--- Optimizing Hyperparameters for LightGBM (scale_pos_weight) ---")

lgbm_base = LGBMClassifier(objective='binary', metric='binary_logloss',
                           random_state=42,
                           scale_pos_weight=scale_pos_weight_value)
param_grid_lgbm = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [20, 31, 40],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}
grid_search_lgbm = GridSearchCV(estimator=lgbm_base,
                               param_grid=param_grid_lgbm,
                               scoring=recall_scorer,
                               cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                               n_jobs=-1,
                               verbose=1)

grid_search_lgbm.fit(X_train, y_train)
print(f"Best Parameters (LightGBM): {grid_search_lgbm.best_params_}")
print(f"Best Recall (LightGBM) on CV: {grid_search_lgbm.best_score_:.4f}")
best_lgbm_model = grid_search_lgbm.best_estimator_
y_pred_lgbm_tuned = best_lgbm_model.predict(X_test)
y_proba_lgbm_tuned = best_lgbm_model.predict_proba(X_test)[:, 1]
print("\n--- Classification Report (LightGBM Tuned) ---")
print(classification_report(y_test, y_pred_lgbm_tuned))
cm_lgbm_tuned = confusion_matrix(y_test, y_pred_lgbm_tuned)
plt.figure(figsize=(6, 5))
sns.heatmap(cm_lgbm_tuned, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Churn (0)', 'Churn (1)'],
            yticklabels=['No Churn (0)', 'Churn (1)'])
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - LightGBM Tuned')
plt.show()
auc_lgbm_tuned = roc_auc_score(y_test, y_proba_lgbm_tuned)
print(f"AUC-ROC (LightGBM Tuned): {auc_lgbm_tuned:.4f}")
fig, ax = plt.subplots(figsize=(6, 5))
RocCurveDisplay.from_estimator(best_lgbm_model, X_test, y_test, ax=ax, name='LightGBM Tuned')
plt.title('ROC Curve - LightGBM Tuned')
plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.legend()
plt.show()

The optimized versions with scale_pos_weight consistently outperform their SMOTETomek-based counterparts in AUC, Recall, and F1-Score.
The conclusion is that, for this problem and dataset, using class_weight and scale_pos_weight in conjunction with hyperparameter optimization was the most effective strategy for achieving high Recall and AUC, surpassing the resampling approach with SMOTETomek.

### Analysis and Comparison of Model Performance Metrics 

###### Results Consolidation:

In [ ]:
# Original Models (Without Optimization)
data_initial = [
    ['Logistic Regression', "class_weight='balanced'", 0.51, 0.78, 0.62, 0.8389, "Inicial"],
    ['Random Forest', "class_weight='balanced'", 0.63, 0.49, 0.55, 0.8195, "Inicial"],
    ['Logistic Regression', "SMOTETomek", 0.53, 0.74, 0.62, 0.8323, "Inicial"],
    ['Random Forest', "SMOTETomek", 0.55, 0.64, 0.59, 0.8167, "Inicial"],
    ['SVM', "SMOTETomek", 0.55, 0.73, 0.62, 0.8233, "Inicial"],
    ['Decision Tree', "SMOTETomek", 0.50, 0.61, 0.55, 0.6958, "Inicial"],
    ['XGBoost', "SMOTETomek", 0.55, 0.68, 0.61, 0.8169, "Inicial"],
    ['LightGBM', "SMOTETomek", 0.54, 0.69, 0.61, 0.8268, "Inicial"],
    ['KNN', "SMOTETomek", 0.48, 0.74, 0.58, 0.7799, "Inicial"]
]

# Tuned Models (Optimized)
data_tuned = [
    ['Logistic Regression', "class_weight='balanced'", 0.49, 0.80, 0.61, 0.8353, "Tuned"],
    ['Random Forest', "class_weight='balanced'", 0.54, 0.74, 0.62, 0.8421, "Tuned"],
    ['SVM', "class_weight='balanced'", 0.46, 0.88, 0.60, 0.8334, "Tuned"],
    ['XGBoost', "scale_pos_weight", 0.49, 0.81, 0.61, 0.8410, "Tuned"],
    ['LightGBM', "scale_pos_weight", 0.52, 0.79, 0.63, 0.8428, "Tuned"]
]

# Consolidate
columns = ['Model', 'Aproach', 'Precision (Churn=1)', 'Recall (Churn=1)', 
           'F1-Score (Churn=1)', 'AUC-ROC', 'Status']
df_results = pd.DataFrame(data_initial + data_tuned, columns=columns)

###### Sorting by Recall (Churn=1):

In [ ]:
df_sorted = df_results.sort_values(by="Recall (Churn=1)", ascending=False).reset_index(drop=True)
display(df_sorted)

###### Visuals: One plot for each metric :

In [ ]:
modelos_com_tuned = ['Logistic Regression', 'Random Forest', 'SVM', 'XGBoost', 'LightGBM']
df_filtered = df_results[df_results['Model'].isin(modelos_com_tuned)]
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
metricas = {
    'Precision (Churn=1)': 'Precision',
    'Recall (Churn=1)': 'Recall',
    'F1-Score (Churn=1)': 'F1-Score',
    'AUC-ROC': 'AUC-ROC'
}
for metrica, titulo in metricas.items():
    ordem = (
        df_filtered[df_filtered["Status"]=="Tuned"]
        .sort_values(by=metrica, ascending=True)["Model"]
        .tolist()
    )
    
    plt.figure(figsize=(12, 7))
    sns.barplot(
        x="Model", y=metrica, hue="Status", 
        data=df_filtered, palette="Set2", order=ordem
    )
    plt.title(f"{titulo} by Model (Initial vs Tuned)", fontsize=16)
    plt.ylabel(titulo)
    plt.xlabel("Model")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

###### Radar Chart:

In [ ]:
df_tuned_only = df_results[df_results['Status'] == "Tuned"].set_index("Model")
metrics = ['Precision (Churn=1)', 'Recall (Churn=1)', 'F1-Score (Churn=1)', 'AUC-ROC']
df_metrics = df_tuned_only[metrics]
df_norm = df_metrics.apply(lambda x: (x - x.min()) / (x.max() - x.min()))
labels = metrics
num_vars = len(labels)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]
plt.figure(figsize=(9, 9))
ax = plt.subplot(111, polar=True)
for idx, row in df_norm.iterrows():
    values = row.tolist()
    values += values[:1]  # close the plot
    ax.plot(angles, values, label=idx, linewidth=2)
    ax.fill(angles, values, alpha=0.15)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(labels, fontsize=12)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(["0.2","0.4","0.6","0.8","1.0"], fontsize=10)
ax.set_ylim(0, 1)
plt.title("Radar Chart - Comparison of Tuned Models", fontsize=16, pad=20)
plt.legend(loc="upper right", bbox_to_anchor=(1.2, 1.1))
plt.show()

Considering the results, the most promising models are:

- Optimized LightGBM: Highest AUC and F1-Score, and an excellent Recall (0.79). It offers the best overall balance.
- Optimized SVM: Highest Recall (0.88), but with the lowest Precision (0.46). It is important to evaluate if the cost of False Positives is justified by the extremely high Recall.

### Treshold Tunning

Now that we have optimized models with excellent metrics, the next step is threshold tuning.

All the models we have used have a default classification threshold of 0.5. This means that if the predicted probability of churn is >= 0.5, the customer is classified as "churn"; otherwise, they are classified as "no churn". However, for our problem, where Recall is the priority and Precision is a secondary concern, the 0.5 threshold may not be optimal.

We need to find a cutoff point that maximizes Recall for the "Churn" class (to avoid missing customers who are going to cancel), while maintaining Precision at an acceptable level (to avoid spending excessive resources on False Positives).


## LightGBM and SVM Tunned

In [ ]:
def plot_pr_curve_and_evaluate_threshold(model, X_test, y_test, model_name):
    y_proba = model.predict_proba(X_test)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_test, y_proba)
    auprc = auc(recall, precision)
    print(f"\n--- Cutoff Point Analysis for: {model_name} ---")
    print(f"Area Under the Precision-Recall Curve (AUPRC): {auprc:.4f}")
    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, marker='.', label=f'{model_name} (AUPRC = {auprc:.2f})')
    plt.xlabel('Recall (Coverage)')
    plt.ylabel('Precision')
    plt.title(f'Precision-Recall Curve - {model_name}')
    plt.legend()
    plt.grid(True)
    plt.show()
    print("\nPrecision and Recall at different Cutoff Points:")
    selected_threshold_indices = np.linspace(0, len(thresholds) - 1, 10).astype(int)
    print("Threshold | Precision  | Recall")
    print("----------------------------")
    for i in selected_threshold_indices:
        idx = min(i, len(thresholds) - 1)
        print(f"{thresholds[idx]:<9.2f} | {precision[idx]:<8.2f}   | {recall[idx]:<6.2f}")

    print("\nEvaluation Example with a Custom Threshold (Default 0.5):")
    custom_threshold = 0.5
    y_pred_custom_threshold = (y_proba >= custom_threshold).astype(int)
    print(f"Classification Report for Threshold = {custom_threshold:.2f}:")
    print(classification_report(y_test, y_pred_custom_threshold))
    cm_custom = confusion_matrix(y_test, y_pred_custom_threshold)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm_custom, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Churn (0)', 'Churn (1)'],
                yticklabels=['No Churn (0)', 'Churn (1)'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title(f'Confusion Matrix - {model_name} (Threshold: {custom_threshold:.2f})')
    plt.show()
# Optimized LightGBM
plot_pr_curve_and_evaluate_threshold(best_lgbm_model, X_test, y_test, "Optimized LightGBM")
# Optimized SVM
plot_pr_curve_and_evaluate_threshold(best_svm_model, X_test, y_test, "Optimized SVM")

### Model Selection

Considering our primary objective to prioritize Recall (capturing the maximum number of churners), without completely disregarding Precision (to avoid wasting resources):

The Optimized LightGBM with the default cutoff point of 0.50 proves to be the most balanced and effective choice.

It achieves a Recall of 0.79, meaning it correctly identifies 79% of the customers who will actually cancel. This is excellent for proactive retention.

It maintains a Precision of 0.52, which indicates that of the customers the model flags as potential churners, a little over half will actually cancel. This implies a manageable cost of False Positives.

Its F1-Score of 0.63 is the highest among the optimized models, confirming its strong balance.

Its AUPRC (0.6565) and AUC-ROC (0.8428) are the highest, attesting to its overall robustness.

The Optimized SVM is impressive for its potential for an extremely high Recall (reaching 0.88 at lower thresholds), but the drop in Precision would be too steep (below 0.50) to justify its selection for this project. This would only be viable if the cost of a False Positive were absolutely negligible, which is rarely the case in retention strategies.

### Conclusion
Chosen Model: Optimized LightGBM Classifier
Treshold: 0.50 (default)

The Optimized LightGBM, using its default cutoff point of 0.50, offers the best balance between detecting customers who will actually cancel (79% Recall) and minimizing unnecessary interventions (52% Precision). This balance is ideal for a business strategy where the priority is to retain customers, ensuring that the majority of potential churners are identified, while simultaneously optimizing the use of retention resources. Its superior performance in AUPRC and AUC-ROC also attests to its overall discriminatory power.

In [ ]:
#| include: false
# salvar o modelo treinado
joblib.dump(best_lgbm_model, "best_lgbm_model.pkl")

## Feature Importance

Feature Importance Analysis seeks to quantify the contribution of each variable (feature) in our dataset to the model's predictions. In other words, it helps us understand which customer characteristics have the greatest impact on the probability of churn, revealing the "triggers" and "protectors" behind this behavior.

For this analysis, we will employ the SHAP (SHapley Additive exPlanations) methodology. SHAP is an advanced and widely respected technique known for its ability to offer transparent and fair interpretability. Unlike simpler methods, SHAP not only tells us "how important" a feature is, but also:

* The Magnitude of the Impact: How much a feature, on average, pushes the churn prediction up or down.

* The Direction of the Impact: Whether high or low values of a feature tend to increase or decrease the probability of churn.

* Individual Context: How each feature affects the churn prediction for each individual customer, allowing for a detailed understanding.

Why is this analysis vital for the Business?

Understanding the importance and directionality of features is fundamental for:

* Strategic Prioritization: Directing marketing, sales, and customer service resources to the areas that truly make a difference in retention.

* Developing Focused Actions: Creating retention campaigns, adjusting product/service offerings, or improving aspects of customer service based on concrete evidence.

* Identifying Opportunities: Discovering new insights into the profile of customers who churn and those who remain loyal.

By the end of this analysis, we will have a clear view of the main drivers of churn, empowering the business to make more informed and proactive decisions to improve the satisfaction and loyalty of our customers.

###### Functions:

In [ ]:
def plot_top_importances(df_imp, value_col, top_n=20, title="Feature Importance"):
    dfp = df_imp.sort_values(value_col, ascending=False).head(top_n)
    plt.figure(figsize=(8, 6))
    plt.barh(dfp["feature"][::-1], dfp[value_col][::-1])
    plt.title(title)
    plt.xlabel(value_col)
    plt.tight_layout()
    plt.show()

def group_by_parent_feature(feature_names):
    parents = []
    for f in feature_names:
        parents.append(f.split('_', 1)[0])
    return np.array(parents)

def aggregate_grouped_importances(values, feature_names, agg="sum"):
    parents = group_by_parent_feature(feature_names)
    df = pd.DataFrame({"feature": feature_names, "value": values, "parent": parents})
    if agg == "sum":
        agg_df = (df.assign(value_abs=lambda d: d["value"].abs())
                    .groupby("parent", as_index=False)["value_abs"].sum()
                    .rename(columns={"parent":"feature", "value_abs":"value"}))
    elif agg == "mean":
        agg_df = (df.assign(value_abs=lambda d: d["value"].abs())
                    .groupby("parent", as_index=False)["value_abs"].mean()
                    .rename(columns={"parent":"feature", "value_abs":"value"}))
    else:
        raise ValueError("agg must be 'sum' or 'mean'")
    return agg_df

###### Gain:

In [ ]:
# feature names
feat_names = X_train.columns
# importance by "gain"
lgbm = joblib.load('best_lgbm_model.pkl')
gain_importances = lgbm.booster_.feature_importance(importance_type="gain")
imp_gain_df = pd.DataFrame({"feature": feat_names, "gain": gain_importances})
imp_gain_df = imp_gain_df.sort_values("gain", ascending=False)
# detailed plot
plot_top_importances(imp_gain_df, "gain", top_n=20, title="LightGBM - Importance (gain)")
# GROUPED version by 'parent' variable
imp_gain_grouped = aggregate_grouped_importances(gain_importances, feat_names, agg="sum")
plot_top_importances(imp_gain_grouped, "value", top_n=15, title="LightGBM - Aggregated Importance (gain)")
imp_gain_grouped.head(15)

###### Permutation Importance:

In [ ]:
perm = permutation_importance(
    lgbm, X_test, y_test,
    n_repeats=10, random_state=42, scoring="roc_auc"
)
imp_perm_df = pd.DataFrame({
    "feature": feat_names,
    "perm_imp_auc": perm.importances_mean,
    "perm_imp_std": perm.importances_std
}).sort_values("perm_imp_auc", ascending=False)

plot_top_importances(imp_perm_df, "perm_imp_auc", top_n=20, title="Permutation Importance (AUC)")

# GROUPED by 'parent' variable
imp_perm_grouped = aggregate_grouped_importances(imp_perm_df["perm_imp_auc"].values, feat_names, agg="sum")
plot_top_importances(imp_perm_grouped, "value", top_n=15, title="Aggregated Permutation Importance (AUC)")
imp_perm_grouped.head(15)

###### SHAP Analysis:

In [ ]:
# Sample
X_sample = X_test.copy()
if X_sample.shape[0] > 2000:
    X_sample = X_sample.sample(2000, random_state=42)

# Explainer for LightGBM
lgbm = joblib.load('best_lgbm_model.pkl')
explainer = shap.TreeExplainer(lgbm)
shap_values = explainer.shap_values(X_sample)

# For binary, shap_values
sv = shap_values[1] if isinstance(shap_values, list) else shap_values

# 5.1) Beeswarm (global)
shap.summary_plot(sv, X_sample, show=True)

# 5.2) Importance by mean |SHAP|
mean_abs = np.abs(sv).mean(axis=0)
imp_shap_df = pd.DataFrame({"feature": X_sample.columns, "mean_abs_shap": mean_abs})\
                 .sort_values("mean_abs_shap", ascending=False)

plot_top_importances(imp_shap_df, "mean_abs_shap", top_n=20, title="SHAP - Mean |impact| (global)")

# 5.3) Directionality
def shap_direction(sv, X):
    dirs = []
    for i, f in enumerate(X.columns):
        xcol = pd.to_numeric(X[f], errors="coerce")
        corr = np.corrcoef(xcol.fillna(xcol.mean()), sv[:, i])[0, 1]
        dirs.append(corr)
    return pd.DataFrame({"feature": X.columns, "direction_corr": dirs})

dir_df = shap_direction(sv, X_sample)

shap_exec_df = (imp_shap_df.merge(dir_df, on="feature")
                            .assign(signal=lambda d: np.where(d["direction_corr"]>=0, "↑ risk", "↓ risk")))

# GROUPED by 'parent' for dummies
shap_group = aggregate_grouped_importances(imp_shap_df["mean_abs_shap"].values, imp_shap_df["feature"].values, agg="sum")
plot_top_importances(shap_group, "value", top_n=15, title="Aggregated SHAP - Mean |impact|")

shap_exec_df.head(20)

###### PDP (Partial Dependence Plots):

In [ ]:
# select top 6 by SHAP global metric
top6 = imp_shap_df["feature"].head(6).tolist()

for f in top6:
    fig = plt.figure(figsize=(6,4))
    PartialDependenceDisplay.from_estimator(lgbm, X_test, [f], kind="average")
    plt.title(f"PDP – {f}")
    plt.tight_layout()
    plt.show()

#### Consolidated Ranking:

In [ ]:
# normalize and combine
r_gain = imp_gain_df.assign(rank_gain=lambda d: d["gain"].rank(ascending=False))
r_perm = imp_perm_df.assign(rank_perm=lambda d: d["perm_imp_auc"].rank(ascending=False))[["feature","rank_perm"]]
r_shap = imp_shap_df.assign(rank_shap=lambda d: d["mean_abs_shap"].rank(ascending=False))[["feature","rank_shap"]]

rank_df = (r_gain[["feature","rank_gain"]]
           .merge(r_perm, on="feature", how="outer")
           .merge(r_shap, on="feature", how="outer")
           .merge(shap_exec_df[["feature","direction_corr","signal"]], on="feature", how="left"))

rank_df["rank_mean"] = rank_df[["rank_gain","rank_perm","rank_shap"]].mean(axis=1)
rank_df = rank_df.sort_values("rank_mean").reset_index(drop=True)

# Show consolidated top 15
rank_df.head(15)

### Recommendations and Business Actions Based on Insights:

* Contract Type – Two-Year Contracts (Contract_Two year): Customers with two-year contracts show a significantly lower risk of churn. Encourage migration to this type of contract through offers and benefits.

* Contract Type – One-Year Contracts (Contract_One year): Customers with one-year contracts also present lower churn risk compared to month-to-month contracts, but higher than two-year contracts. Reinforce the value of renewal and promote the transition to longer-term contracts.

* Tenure: Customers with short tenure are at higher risk. Implement robust onboarding programs and proactive follow-up during the first months to ensure satisfaction and early engagement.

* Internet Service – Fiber Optic (InternetService_Fiber optic): The fact that fiber optic customers show higher churn risk is a red flag. Service quality, support, or perceived cost-benefit for fiber users should be urgently investigated. Potential issues may include performance problems or inadequate pricing in this segment.

* Monthly Charges (MonthlyCharges): Higher monthly charges increase churn risk. Analyze the price ranges where churn is most prevalent. Consider offering bundled packages or renegotiation for customers with high charges, especially when combined with low tenure.

* Payment Method – Electronic Check (PaymentMethod_Electronic check): Customers using electronic checks are at high risk. This may indicate a segment with lower loyalty, financial difficulties, or dissatisfaction. Consider incentives to switch to other payment methods or establish closer monitoring for these customers.

* Online Security (OnlineSecurity): Not having online security services increases churn risk. Promote the importance and benefits of online security offerings.

* Streaming Services (StreamingMovies, StreamingTV): Having streaming services may increase churn risk. This is counterintuitive and may suggest that these services are not adding enough value to retain customers, or that customers seeking such services tend to have a profile more prone to exploring alternatives.


# Deploy

Check this model in production by running churn predictions:

<a href="https://ds-churn-prediction-ml-deploy-qp3znuqfw4fpwqcv8li3n2.streamlit.app/" 
   target="_blank" 
   rel="noopener noreferrer">
  Click here to access
</a>